In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import RMSprop
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import load_model
from keras.utils import get_custom_objects

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units

In [7]:
def acc_model(filename):
    # load data
    df = pd.read_csv(filename, delimiter=';')
    # Menentukan input and output
    X = df.drop(columns=["GOAL","7MSHOT","9MSHOT","6MSHOT","WINGSHOT","PIVOTSHOT","FASTBREAKSHOT"])
    y = df.GOAL
    # Membagi data set 80 : 20 train dan test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Merancang Neural Network Model
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))  # Adjust the input shape to (12,)
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))  # Use 'linear' activation for regression
    model.compile(optimizer='rmsprop', loss=mean_squared_error,metrics=['mean_squared_error', 'mean_absolute_error'])

    # Custom callback to collect metrics for plotting
    class MetricsCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            self.mse_values.append(logs['mean_squared_error'])
            self.mae_values.append(logs['mean_absolute_error'])

        def on_train_begin(self, logs=None):
            self.mse_values = []
            self.mae_values = []
        
    metrics_callback = MetricsCallback()
    history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test),callbacks=[metrics_callback], verbose=0)
    #model.save(f'modelacc_{os.path.splitext(os.path.basename(filename))[0]}.h5')

    # data prediction
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    def mean_squared_error1(y_test, pred_test):
        squared_errors = [(true - pred)**2 for true, pred in zip(y_test, pred_test)]
        mse = sum(squared_errors) / len(y_test)
        return mse
    
    mse = mean_squared_error1(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    mae = mean_absolute_error(y_test, pred_test)
    print("Manual MSE:", mse)
    print(f"Mean Absolute Error (MAE)",mae)  
    print(f"R2 Score",r2)
    
    return model
def shot_model(filename):
    # load data
    df = pd.read_csv(filename, delimiter=';')
    # Menentukan input and output
    X = df.drop(columns=["GOAL","7MACC","9MACC","6MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
    y = df.GOAL
    # Membagi data set 80 : 20 train dan test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Normalisasi data
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # Merancang Neural Network Model
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))  # Adjust the input shape to (12,)
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))  # Use 'linear' activation for regression
    model.compile(optimizer='rmsprop', loss=mean_squared_error,metrics=['mean_squared_error', 'mean_absolute_error'])

    # Custom callback to collect metrics for plotting
    class MetricsCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            self.mse_values.append(logs['mean_squared_error'])
            self.mae_values.append(logs['mean_absolute_error'])

        def on_train_begin(self, logs=None):
            self.mse_values = []
            self.mae_values = []
        
    metrics_callback = MetricsCallback()
    history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test),callbacks=[metrics_callback], verbose=0)
    #model.save(f'modelshott_{os.path.splitext(os.path.basename(filename))[0]}.h5')
   
     # data prediction
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    def mean_squared_error1(y_test, pred_test):
        squared_errors = [(true - pred)**2 for true, pred in zip(y_test, pred_test)]
        mse = sum(squared_errors) / len(y_test)
        return mse
    
    mse = mean_squared_error1(y_test, pred_test)
    r2 = r2_score(y_test, pred_test)
    mae = mean_absolute_error(y_test, pred_test)
    print("Manual MSE:", mse)
    print(f"Mean Absolute Error (MAE)",mae)  
    print(f"R2 Score",r2)
    
    

    return model

In [8]:
for i in os.listdir('data/'):
    print('Position: ', i.split(',')[0])
    acc_model('data/'+i)
    print('')
for u in os.listdir('data/'):
    print('Position: ', u.split(',')[0])
    shot_model('data/'+u)
    print('')

Position:  CB.csv
9/9 [==============================] - 0s 1ms/step
Manual MSE: [1.4643815]
Mean Absolute Error (MAE) 0.8040318978849308
R2 Score 0.6973650738021899

Position:  LB.csv
10/10 [==============================] - 0s 997us/step
Manual MSE: [1.2978475]
Mean Absolute Error (MAE) 0.7603026350207677
R2 Score 0.7211833091122786

Position:  LW.csv
8/8 [==============================] - 0s 1ms/step
Manual MSE: [1.4121455]
Mean Absolute Error (MAE) 0.7514436561128367
R2 Score 0.7970641697259992

Position:  P.csv
10/10 [==============================] - 0s 2ms/step
Manual MSE: [1.0916319]
Mean Absolute Error (MAE) 0.6329890231486239
R2 Score 0.6550947560655056

Position:  RB.csv
7/7 [==============================] - 0s 1ms/step
Manual MSE: [1.1948366]
Mean Absolute Error (MAE) 0.7228784935040907
R2 Score 0.795077145596279

Position:  RW.csv
8/8 [==============================] - 0s 1ms/step
Manual MSE: [1.4064187]
Mean Absolute Error (MAE) 0.764644816129104
R2 Score 0.7557306185933